In [1]:
%matplotlib inline
import math
import numpy as np
from matplotlib import pyplot as plt

## Using Python Libraries (SciPy) for Integration
While learning basic rules (Riemann sums, trapezoid, Simpson) is essential, in practice you often rely on library routines that are:
- More accurate (adaptive step size)
- More convenient (automatic error estimates)
- Able to handle infinite limits / oscillatory behavior

### 1. `scipy.integrate.quad`
High–quality adaptive integrator (wraps QUADPACK). Usage:
```python
from scipy.integrate import quad
val, err_est = quad(f, a, b)
```
Gives an *estimate* `val` and an *absolute error bound* `err_est` (heuristic but usually reliable). You can supply `epsabs` (absolute tol) and `epsrel` (relative tol).

Handles improper integrals:
```python
quad(f, 0, np.inf)    # infinite upper limit
quad(f, -np.inf, np.inf)
```

### 2. Vectorized Simpson / Trapezoid
If you already have sampled data values on a uniform or non-uniform grid:
```python
from scipy.integrate import simpson, trapezoid
simpson(y, x=x)      # Simpson (needs odd number of points)
trapezoid(y, x=x)    # Trapezoid
```
Both accept just `y` with implied spacing `dx` if uniform.

### 3. When to Use What?
| Situation | Recommended |
|-----------|-------------|
| Smooth function with cheap evaluations | `quad` (adaptive) |
| Already have tabulated data | `simpson` (or `trapezoid` if only few points) |
| Need very high precision | Increase `epsabs`, `epsrel` in `quad` or refine grid and use higher-order rules |
| Infinite / semi-infinite interval | `quad` with `np.inf` bounds |

### 4. Error Awareness
- `quad` returns an error *estimate* (not a strict bound, but rarely far off for well-behaved functions).
- `simpson`/`trapezoid` do NOT give an error estimate: you refine the grid and compare successive results.

### 5. Strategy to Trust a Result
1. Compute with a chosen method.
2. Refine (halve `h` or tighten tolerances) and recompute.
3. If results stabilize to desired digits, accept.

Next: a code example comparing manual rules with `quad`.

---

### Advanced: Adjusting Tolerances and Double Integrals

`quad` lets you tighten or loosen accuracy:
```python
val, err = quad(f, a, b, epsabs=1e-10, epsrel=1e-10)
```
- `epsabs`: absolute error target.
- `epsrel`: relative error target.
The algorithm stops when estimated absolute error ≤ max(epsabs, epsrel*|val|).

#### Double Integrals with `dblquad`
For an integral
$$ I = \int_{x=a}^{b} \int_{y=g_1(x)}^{g_2(x)} F(x,y)\,dy\,dx, $$
use
```python
from scipy.integrate import dblquad
I, err = dblquad(lambda y,x: F(x,y), a, b, g1, g2)
```
Note argument order: inner variable first in the lambda.

If limits are simple constants you can also construct tensor grids and use Simpson/trapezoid in 2D, but adaptive `dblquad` handles curved boundaries cleanly.

#### Higher Dimensions
- `nquad` (general n-D with lists of bounds / callables)
- Monte Carlo (random sampling) when dimension is large or region is irregular.

Below: examples of refined `quad` tolerance and a double integral.

---

### Example / Practice (Library Integration)
Answer the following using SciPy:


---

1. Use `quad` to approximate $I_1 = \int_0^1 e^{-x}\sin(x^2)\,dx$.
   - Record the returned value and error estimate.
   - Tighten tolerances to `epsabs=epsrel=1e-12`; how many more (or fewer) function evaluations does it report (see `quad`'s `full_output` if desired)?


In [2]:
import numpy as np
from scipy.integrate import quad

# Define the integrand for the first integral
def f(x):
    return np.exp(-x) * np.sin(x**2)

# --- Part 1: Default Tolerances ---
# Use full_output=True to get the number of function evaluations
result1_default, error1_default, info1_default = quad(f, 0, 1, full_output=True)

print("--- Default Tolerances ---")
print(f"Approximate value of I_1: {result1_default:.12f}")
print(f"Estimated error: {error1_default:.2e}")
print(f"Number of function evaluations: {info1_default['neval']}")
print("-" * 28)

# --- Part 2: Tighter Tolerances ---
# Set absolute and relative error tolerances to 1e-12
result1_tight, error1_tight, info1_tight = quad(f, 0, 1, epsabs=1e-12, epsrel=1e-12, full_output=True)

print("\n--- Tighter Tolerances (epsabs=1e-12, epsrel=1e-12) ---")
print(f"Approximate value of I_1: {result1_tight:.12f}")
print(f"Estimated error: {error1_tight:.2e}")
print(f"Number of function evaluations: {info1_tight['neval']}")

# --- Comparison ---
eval_diff = info1_tight['neval'] - info1_default['neval']
print(f"\nChange in function evaluations: {eval_diff}")

--- Default Tolerances ---
Approximate value of I_1: 0.150912567223
Estimated error: 1.68e-15
Number of function evaluations: 21
----------------------------

--- Tighter Tolerances (epsabs=1e-12, epsrel=1e-12) ---
Approximate value of I_1: 0.150912567223
Estimated error: 1.68e-15
Number of function evaluations: 21

Change in function evaluations: 0


---

2. Evaluate the improper integral $I_2 = \int_0^{\infty} e^{-x}\,dx$ with `quad` and verify the value.


In [7]:

# Define the integrand for the second integral
def g(x):
  return np.exp(-x)

# --- Numerical Evaluation ---
# Use np.inf for the upper limit of integration
result2, error2 = quad(g, 0, np.inf)

print("--- Numerical Evaluation using SciPy quad ---")
print(f"Approximate value of I_2: {result2}")
print(f"Estimated error: {error2:.2e}")

# --- Analytical Verification ---
# The analytical solution is:
# integral(e^-x) from 0 to infinity = [-e^-x] from 0 to infinity
# = lim(b->inf) [-e^-b] - [-e^-0]
# = 0 - (-1) = 1
analytical_solution = 1.0
print(f"\n--- Analytical Verification ---")
print(f"The exact value of the integral is: {analytical_solution}")


--- Numerical Evaluation using SciPy quad ---
Approximate value of I_2: 1.0000000000000002
Estimated error: 5.84e-11

--- Analytical Verification ---
The exact value of the integral is: 1.0


---

3. Compute the triangular-region double integral
   $$ I_3 = \int_{x=0}^{1} \int_{y=0}^{x} (x^2 + y^2)\,dy\,dx $$
   and compare with the exact $1/3$.


In [10]:
import numpy as np
from scipy.integrate import dblquad

# --- Numerical Calculation ---
# Define the integrand f(y, x). Note: dblquad expects the inner variable (y) first.
def integrand3(y, x):
    return x**2 + y**2

# The outer integral is over x, from 0 to 1.
x_lower = 0
x_upper = 1

# The inner integral is over y. Its limits can be functions of x.
# y goes from 0 to x.
y_lower = 0
y_upper = lambda x: x

# Perform the double integration
result3, error3 = dblquad(integrand3, x_lower, x_upper, y_lower, y_upper)

# --- Comparison ---
exact_value = 1/3

print("--- Triangular-Region Double Integral ---")
print(f"Numerical result: {result3:.8f}")
print(f"Estimated error: {error3:.2e}")
print(f"Exact value:      {exact_value:.8f}")


--- Triangular-Region Double Integral ---
Numerical result: 0.33333333
Estimated error: 1.47e-14
Exact value:      0.33333333


---

4. (Exploration) Change the curved-boundary example to $\int_0^1 \int_0^{x^2} \cos(xy)\,dy\,dx$; estimate its value.


In [6]:
import numpy as np
from scipy.integrate import dblquad

# --- Numerical Calculation ---
# Define the integrand f(y, x).
def integrand4(y, x):
    return np.cos(x * y)

# The outer integral is over x, from 0 to 1.
x_lower = 0
x_upper = 1

# The inner integral is over y, from 0 to x^2.
y_lower = 0
y_upper = lambda x: x**2

# Perform the double integration
result4, error4 = dblquad(integrand4, x_lower, x_upper, y_lower, y_upper)

print("--- Curved-Boundary Double Integral ---")
print(f"Estimated value: {result4:.8f}")
print(f"Estimated error: {error4:.2e}")

--- Curved-Boundary Double Integral ---
Estimated value: 0.31536102
Estimated error: 9.32e-15
